In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install --upgrade transformers tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 63.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [4]:
tokenizer

PreTrainedTokenizerFast(name_or_path='Bllossom/llama-3.2-Korean-Bllossom-3B', vocab_size=128000, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|finetune_right_pad_id|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128005: AddedToken("<|reserved_special_token_2|

In [11]:
help(model.generate)

Help on method generate in module transformers.generation.utils:

generate(inputs: Optional[torch.Tensor] = None, generation_config: Optional[transformers.generation.configuration_utils.GenerationConfig] = None, logits_processor: Optional[transformers.generation.logits_process.LogitsProcessorList] = None, stopping_criteria: Optional[transformers.generation.stopping_criteria.StoppingCriteriaList] = None, prefix_allowed_tokens_fn: Optional[Callable[[int, torch.Tensor], List[int]]] = None, synced_gpus: Optional[bool] = None, assistant_model: Optional[ForwardRef('PreTrainedModel')] = None, streamer: Optional[ForwardRef('BaseStreamer')] = None, negative_prompt_ids: Optional[torch.Tensor] = None, negative_prompt_attention_mask: Optional[torch.Tensor] = None, **kwargs) -> Union[transformers.generation.utils.GenerateDecoderOnlyOutput, transformers.generation.utils.GenerateEncoderDecoderOutput, transformers.generation.utils.GenerateBeamDecoderOnlyOutput, transformers.generation.utils.GenerateBe

In [5]:
instruction = "철수가 20개의 연필을 가지고 있었는데 영희가 절반을 가져가고 민수가 남은 5개를 가져갔으면 철수에게 남은 연필의 갯수는 몇개인가요?"

messages = [
    {"role": "user", "content": f"{instruction}"}
    ]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=1024,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

print(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


철수가 20개의 연필을 가지고 있었고, 영희가 절반을 가져가게 되면, 영희가 가져간 연필의 갯수는 20 / 2 = 10개입니다.

이제 철수가 남은 연필의 갯수는 20 - 10 = 10개입니다. 이후 민수가 남은 5개를 가져갔으므로, 철수가 남은 연필의 갯수는 10 - 5 = 5개입니다.

따라서 철수가 남은 연필의 갯수는 5개입니다.


In [6]:
!ls -al /root/.cache/huggingface/hub/

total 20
drwxr-xr-x 4 root root 4096 Dec 14 09:47 .
drwxr-xr-x 3 root root 4096 Dec 14 09:47 ..
drwxr-xr-x 3 root root 4096 Dec 14 09:47 .locks
drwxr-xr-x 6 root root 4096 Dec 14 09:50 models--Bllossom--llama-3.2-Korean-Bllossom-3B
-rw-r--r-- 1 root root    1 Dec 14 09:47 version.txt


In [7]:
import os

hf_cache_dir = os.path.expanduser("~/.cache/huggingface/hub/")

model_path = "models--Bllossom--llama-3.2-Korean-Bllossom-3B"
model_cache_dir = os.path.join(hf_cache_dir, model_path)

for root, dirs, files in os.walk(model_cache_dir):
    for file in files:
        if file == "tokenizer_config.json":
            config_file_path = root
            break

print("Config file path:", config_file_path)

Config file path: /root/.cache/huggingface/hub/models--Bllossom--llama-3.2-Korean-Bllossom-3B/snapshots/5b51ef02f4a41840db119dc8650696466c645d21


In [8]:
!ls -al /root/.cache/huggingface/hub/models--Bllossom--llama-3.2-Korean-Bllossom-3B/snapshots/5b51ef02f4a41840db119dc8650696466c645d21

total 20
drwxr-xr-x 2 root root 4096 Dec 14 09:50 .
drwxr-xr-x 3 root root 4096 Dec 14 09:47 ..
lrwxrwxrwx 1 root root   52 Dec 14 09:47 config.json -> ../../blobs/375c5eb63befa4d5858bcde9b3dc90704ff249b4
lrwxrwxrwx 1 root root   52 Dec 14 09:50 generation_config.json -> ../../blobs/231d9dcaef9e997baa11b6bb2155d9a8911a83bc
lrwxrwxrwx 1 root root   76 Dec 14 09:49 model-00001-of-00002.safetensors -> ../../blobs/b24e4dcb20e93118fd4299801895a687d3ad829a6a06f743670fa2ff92751290
lrwxrwxrwx 1 root root   76 Dec 14 09:49 model-00002-of-00002.safetensors -> ../../blobs/ce906eb61f62cf527d033df9249f02a425ccd9695621b6684650ec48b2f5c36b
lrwxrwxrwx 1 root root   52 Dec 14 09:47 model.safetensors.index.json -> ../../blobs/d3a1f0f5f401eeadca0c7a6786bd9e877fd42e58
lrwxrwxrwx 1 root root   52 Dec 14 09:47 special_tokens_map.json -> ../../blobs/02ee80b6196926a5ad790a004d9efd6ab1ba6542
lrwxrwxrwx 1 root root   52 Dec 14 09:47 tokenizer_config.json -> ../../blobs/44e477a188a53f034bb31c6b3b18c2f9ce992b2e
l

In [9]:
tokenizer_config_json = os.path.join(config_file_path, 'tokenizer_config.json')
tokenizer_json = os.path.join(config_file_path, 'tokenizer.json')
special_tokens_map_json = os.path.join(config_file_path, 'special_tokens_map.json')
config_json = os.path.join(config_file_path, 'config.json')
generation_config_json = os.path.join(config_file_path, 'generation_config.json')

In [10]:
import json

with open(special_tokens_map_json, "r") as f:
    content = json.load(f)

print(json.dumps(content, indent=2))

{
  "bos_token": {
    "content": "<|begin_of_text|>",
    "lstrip": false,
    "normalized": false,
    "rstrip": false,
    "single_word": false
  },
  "eos_token": {
    "content": "<|eot_id|>",
    "lstrip": false,
    "normalized": false,
    "rstrip": false,
    "single_word": false
  }
}


In [12]:
with open(generation_config_json, "r") as f:
    content = json.load(f)

print(json.dumps(content, indent=2))

{
  "_from_model_config": true,
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128001,
  "temperature": 0.6,
  "top_p": 0.9,
  "transformers_version": "4.45.0"
}
